In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
import pickle
import os
import shutil

from rdkit import Chem
from rdkit import RDLogger 
RDLogger.DisableLog('rdApp.*')
import torch
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, Module, Sequential
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool



c:\Project\Thermophysical Property Melting Point\.venv\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
print("--- Step 1: Defining Data Processing for Hybrid Model ---")

train_tabular = pd.read_csv('../data/processed/train_processed.csv')
test_tabular = pd.read_csv('../data/processed/test_processed.csv')

train_raw = pd.read_csv('../data/raw/train.csv')
test_raw = pd.read_csv('../data/raw/test.csv')

train_merged = pd.merge(train_raw[['id', 'SMILES']], train_tabular, on='id')
test_merged = pd.merge(test_raw[['id', 'SMILES']], test_tabular, on='id')

--- Step 1: Defining Data Processing for Hybrid Model ---


In [3]:
rdkit_features = [col for col in train_tabular.columns if not col.startswith('Group_') and col not in ['id', 'Tm']]
X_train_rdkit = train_merged[rdkit_features]
X_test_rdkit = test_merged[rdkit_features]

rdkit_scaler = StandardScaler()
X_train_rdkit_scaled = rdkit_scaler.fit_transform(X_train_rdkit)
X_test_rdkit_scaled = rdkit_scaler.transform(X_test_rdkit)

with open('../models/rdkit_scaler.pkl', 'wb') as f:
    pickle.dump(rdkit_scaler, f)

In [4]:
def smiles_to_graph(smiles_string, y_val=0, rdkit_feats=None):
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is None: return None
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append([
            atom.GetAtomicNum(), atom.GetFormalCharge(), atom.GetHybridization(),
            atom.GetIsAromatic(), atom.GetTotalNumHs(), atom.GetTotalValence(),
        ])
    x = torch.tensor(atom_features_list, dtype=torch.float)
    edge_indices, edge_attrs = [], []
    for bond in mol.GetBonds():
        i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        bond_type = bond.GetBondTypeAsDouble()
        edge_indices.extend([(i, j), (j, i)])
        edge_attrs.extend([[bond_type], [bond_type]])
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attrs, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor([y_val], dtype=torch.float))
    if rdkit_feats is not None:
        data.rdkit_features = torch.tensor([rdkit_feats], dtype=torch.float)
    return data


In [5]:
class HybridDataset(Dataset):
    def __init__(self, root, data_df, rdkit_features_scaled, test=False, node_feature_scaler=None):
        self.data_df = data_df
        self.rdkit_features_scaled = rdkit_features_scaled
        self.test = test
        self.node_feature_scaler = node_feature_scaler
        super(HybridDataset, self).__init__(root)
        
    @property
    def raw_file_names(self): return []
    @property
    def processed_file_names(self): return [f'{"test" if self.test else "train"}_hybrid.pt']
    def download(self): pass

    def process(self):
        if not self.test and self.node_feature_scaler is None:
            all_node_features = []
            for smiles in tqdm(self.data_df['SMILES'], desc="Fitting Node Scaler"):
                mol = Chem.MolFromSmiles(smiles)
                if mol:
                    for atom in mol.GetAtoms():
                        all_node_features.append([atom.GetAtomicNum(), atom.GetFormalCharge(), atom.GetHybridization(), atom.GetIsAromatic(), atom.GetTotalNumHs(), atom.GetTotalValence()])
            self.node_feature_scaler = StandardScaler()
            self.node_feature_scaler.fit(all_node_features)
            with open('../models/gnn_node_scaler.pkl', 'wb') as f:
                pickle.dump(self.node_feature_scaler, f)

        graphs = []
        for idx, row in tqdm(self.data_df.iterrows(), total=self.data_df.shape[0], desc="Processing SMILES for Hybrid Model"):
            y_val = np.log(row['Tm']) if not self.test else 0
            rdkit_feats = self.rdkit_features_scaled[idx]
            graph = smiles_to_graph(row['SMILES'], y_val, rdkit_feats)
            if graph is not None:
                graph.x = torch.tensor(self.node_feature_scaler.transform(graph.x), dtype=torch.float)
                graphs.append(graph)
        torch.save(graphs, self.processed_paths[0], pickle_protocol=5)

    def len(self):
        if not hasattr(self, 'graphs'):
            self.graphs = torch.load(self.processed_paths[0], weights_only=False)
        return len(self.graphs)
    def get(self, idx):
        if not hasattr(self, 'graphs'):
            self.graphs = torch.load(self.processed_paths[0], weights_only=False)
        return self.graphs[idx]


In [6]:
print("Instantiating Hybrid Datasets...")
train_dataset = HybridDataset(root='../data/processed/gnn_hybrid', data_df=train_merged, rdkit_features_scaled=X_train_rdkit_scaled)
with open('../models/gnn_node_scaler.pkl', 'rb') as f:
    node_scaler = pickle.load(f)
test_dataset = HybridDataset(root='../data/processed/gnn_hybrid', data_df=test_merged, rdkit_features_scaled=X_test_rdkit_scaled, test=True, node_feature_scaler=node_scaler)
print("Hybrid datasets created.")

Instantiating Hybrid Datasets...
Hybrid datasets created.


In [7]:
print("\n--- Step 2: Defining the Hybrid GNN-MLP Architecture ---")

class HybridGNN(torch.nn.Module):
    def __init__(self, num_node_features, num_rdkit_features, hidden_channels=128):
        super(HybridGNN, self).__init__()
        torch.manual_seed(42)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.bn1 = BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels * 2)
        self.bn2 = BatchNorm1d(hidden_channels * 2)
        
        self.mlp = Sequential(
            Linear(hidden_channels * 2 + num_rdkit_features, hidden_channels * 2), 
            torch.nn.ReLU(),
            BatchNorm1d(hidden_channels * 2),
            Linear(hidden_channels * 2, hidden_channels),
            torch.nn.ReLU(),
            BatchNorm1d(hidden_channels),
            Linear(hidden_channels, 1)
        )

    def forward(self, data):
        x, edge_index, batch, rdkit_feats = data.x, data.edge_index, data.batch, data.rdkit_features
        x = self.conv1(x, edge_index).relu()
        x = self.bn1(x)
        x = self.conv2(x, edge_index).relu()
        x = self.bn2(x)
        graph_embedding = global_mean_pool(x, batch)
        
        combined_features = torch.cat([graph_embedding, rdkit_feats], dim=1)
        
        # Final prediction
        return self.mlp(combined_features).squeeze()



--- Step 2: Defining the Hybrid GNN-MLP Architecture ---


In [8]:
# 3. Training and Submission
print("\n--- Step 3: Starting Training for Hybrid Model ---")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

torch.manual_seed(42)
shuffled_dataset = train_dataset.shuffle()
train_size = int(0.85 * len(shuffled_dataset))
train_data, val_data = shuffled_dataset[:train_size], shuffled_dataset[train_size:]
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)

model = HybridGNN(
    num_node_features=train_dataset.num_node_features,
    num_rdkit_features=X_train_rdkit_scaled.shape[1]
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=10, min_lr=1e-6)
criterion = torch.nn.L1Loss()



--- Step 3: Starting Training for Hybrid Model ---


In [9]:
from sklearn.metrics import mean_absolute_error

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def evaluate(loader):
    """Evaluates the model on a given data loader."""
    model.eval()
    all_preds = []
    all_reals = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            preds = np.exp(out.cpu().numpy())
            reals = np.exp(data.y.cpu().numpy())
            all_preds.extend(preds)
            all_reals.extend(reals)
            
    return mean_absolute_error(all_reals, all_preds)



best_val_mae = float('inf')
patience_counter = 0
patience = 25

for epoch in range(1, 201):
    loss = train()
    val_mae = evaluate(val_loader)
    scheduler.step(val_mae)
    
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        torch.save(model.state_dict(), 'best_hybrid_model.pt')
        patience_counter = 0
    else:
        patience_counter += 1
        
    lr = optimizer.param_groups[0]['lr']
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val MAE: {val_mae:.4f}, Best Val MAE: {best_val_mae:.4f}, LR: {lr:.6f}')
        
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch}.")
        break


Epoch: 001, Loss: 1.6693, Val MAE: 4.1955, Best Val MAE: 4.1955, LR: 0.000500
Epoch: 002, Loss: 1.4933, Val MAE: 3.5116, Best Val MAE: 3.5116, LR: 0.000500
Epoch: 003, Loss: 1.2206, Val MAE: 3.2386, Best Val MAE: 3.2386, LR: 0.000500
Epoch: 004, Loss: 0.8361, Val MAE: 2.7251, Best Val MAE: 2.7251, LR: 0.000500
Epoch: 005, Loss: 0.4351, Val MAE: 1.6005, Best Val MAE: 1.6005, LR: 0.000500
Epoch: 006, Loss: 0.1982, Val MAE: 0.9245, Best Val MAE: 0.9245, LR: 0.000500
Epoch: 007, Loss: 0.1437, Val MAE: 0.5808, Best Val MAE: 0.5808, LR: 0.000500
Epoch: 008, Loss: 0.0968, Val MAE: 0.5036, Best Val MAE: 0.5036, LR: 0.000500
Epoch: 009, Loss: 0.0881, Val MAE: 0.3526, Best Val MAE: 0.3526, LR: 0.000500
Epoch: 010, Loss: 0.0667, Val MAE: 0.3471, Best Val MAE: 0.3471, LR: 0.000500
Epoch: 011, Loss: 0.0625, Val MAE: 0.3524, Best Val MAE: 0.3471, LR: 0.000500
Epoch: 012, Loss: 0.0545, Val MAE: 0.3068, Best Val MAE: 0.3068, LR: 0.000500
Epoch: 013, Loss: 0.0622, Val MAE: 0.4561, Best Val MAE: 0.3068,

In [10]:
print("\n--- Generating Final Submission from Hybrid Model ---")
model.load_state_dict(torch.load('best_hybrid_model.pt'))
test_preds = []
model.eval()
with torch.no_grad():
    for data in tqdm(test_loader, desc="Predicting with Hybrid Model"):
        data = data.to(device)
        out = model(data)
        test_preds.extend(np.exp(out.cpu().numpy()))

submission_df = pd.read_csv('../data/raw/sample_submission.csv')
submission_df['Tm'] = test_preds
submission_df.to_csv('submission_hybrid_gnn.csv', index=False)
print("Submission file 'submission_hybrid_gnn.csv' created successfully!")
print(submission_df.head())



--- Generating Final Submission from Hybrid Model ---


Predicting with Hybrid Model:   0%|          | 0/11 [00:00<?, ?it/s]

Submission file 'submission_hybrid_gnn.csv' created successfully!
     id        Tm
0  1022  5.793273
1  1146  5.745530
2    79  5.348372
3  2279  5.332273
4  1342  5.318476


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error
import torch

def evaluate_in_kelvin(loader, model, device):
    """
    Evaluates the model and converts predictions back to original Kelvin scale.
    """
    model.eval()
    all_preds_kelvin = []
    all_reals_kelvin = []
    
    print("Evaluating in Kelvin...")
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            
            preds_log1p = np.exp(out.cpu().numpy())
            reals_log1p = np.exp(data.y.cpu().numpy())
            
            # Inverse the Feature Engineering log1p transformation (get back to Kelvin)
            preds_kelvin = np.expm1(preds_log1p)
            reals_kelvin = np.expm1(reals_log1p)
            
            all_preds_kelvin.extend(preds_kelvin)
            all_reals_kelvin.extend(reals_kelvin)
            
    # Calculate MAE on the true Kelvin scale
    mae = mean_absolute_error(all_reals_kelvin, all_preds_kelvin)
    return mae, all_preds_kelvin

# Calculate and print the true MAE
true_mae, _ = evaluate_in_kelvin(val_loader, model, device)
print(f"\n Final Calibrated Result (Hybrid GNN): {true_mae:.4f} Kelvin")

Evaluating in Kelvin...

 Final Calibrated Result (Hybrid GNN): 30.2785 Kelvin
